# Stock Market News and Sentiment Analysis Backtesting

In [65]:
# Libraries
from eod import EodHistoricalData
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Importing and assigning the api key
with open("../../eodHistoricalData-API.txt", "r") as f:
    api_key = f.read()
    
# EOD Historical Data client
client = EodHistoricalData(api_key)

## Retrieving News

In [13]:
def getNews(ticker, days):
    """
    Retrieves financial news over the course of a specified number of days for
    a given stock ticker.
    """

    # List of news
    news = []
    
    # How many days back to retrieve
    ago = datetime.now() - timedelta(days=days)

    # Getting news over the course of a year
    for i in tqdm(range(10, days, 10)):

        # Grabbing the news
        resp = client.get_financial_news(
            s=ticker,
            from_=(ago+timedelta(days=i-10)).strftime("%Y-%m-%d"),
            to=(ago+timedelta(days=i)).strftime("%Y-%m-%d"),
            limit=50
        )

        # Adding to the news list
        news.extend(resp)
        
    # Filtering out irrelevant news
    lst = [i for i in news if sum(x in {ticker.lower()} for x in nltk.word_tokenize(i['title'].lower()))>=1]
    
    return pd.DataFrame(lst)

In [14]:
# Retrieving financial news
news = getNews("AMD", 365)

100%|██████████| 36/36 [00:22<00:00,  1.59it/s]


## Retrieving Price History

In [10]:
# Date
ago = datetime.now() - timedelta(days=365)

# Prices
prices = pd.DataFrame(client.get_prices_eod("AMD", from_=ago.strftime("%Y-%m-%d")))

In [11]:
prices

,date,open,high,low,close,adjusted_close,volume
0,2020-11-30,87.33,92.7400,86.5300,92.66,92.66,84482979
1,2020-12-01,92.25,93.9000,90.7800,92.63,92.63,58670527
2,2020-12-02,92.89,96.3700,92.5253,93.74,93.74,57988822
3,2020-12-03,94.06,94.7000,92.0100,92.31,92.31,35859716
4,2020-12-04,92.58,94.5800,90.6300,94.04,94.04,45570273
...,...,...,...,...,...,...,...
246,2021-11-19,155.76,156.9200,153.4450,155.41,155.41,41668862
247,2021-11-22,157.14,161.8799,152.3900,152.52,152.52,58674078
248,2021-11-23,150.41,152.6600,145.2996,149.92,149.92,62852705
249,2021-11-24,149.46,157.9300,147.1900,157.80,157.80,60677062


## Sentiment Analysis

In [17]:
# Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

In [58]:
news['sentiment'] = news['title'].apply(lambda x: sia.polarity_scores(x)['pos'])

In [59]:
news[['sentiment', 'title']].tail(50).loc[389]

sentiment                                                  0.0
title        Here's the Latest Reason AMD Stock Is a Scream...
Name: 389, dtype: object

In [42]:
news['content'][3].strip()

'SANTA CLARA, Calif., Jan.  13, 2021  (GLOBE NEWSWIRE) -- AMD (NASDAQ: AMD) announced today that it will report fourth quarter and fiscal year 2020 financial results on Tuesday, January 26, 2021 after the close of market. Management will conduct a conference call to discuss these results at 5:00 p.m. EST / 2:00 p.m. PST. Interested parties are invited to listen to the webcast of the conference call via AMD’s Investor Relations website ir.amd.com. An archive of the webcast will be available for approximately one year after the conference call.\n\nAbout AMD \r\nFor more than 50 years, AMD has driven innovation in high-performance computing, graphics and visualization technologies ― the building blocks for gaming, immersive platforms and the data center. Hundreds of millions of consumers, leading Fortune 500 businesses and cutting-edge scientific research facilities around the world rely on AMD technology daily to improve how they live, work and play. AMD employees around the world are fo